In [1]:
# 경고창 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import pymongo
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats
from konlpy.tag import Okt
okt = Okt()
from mecab import MeCab
mecab = MeCab()
import pyLDAvis
import pyLDAvis.lda_model
     
client=pymongo.MongoClient("192.168.10.240",27017)
db= client['AI_LKJ']
collection_tour_train = db['data_tour_NLP_train']
collection_tour = db['tour_contents']
data_tour = collection_tour.find()
data_tour_train = collection_tour_train.find()
df_tour = pd.DataFrame(data_tour)
df_tour_train = pd.DataFrame(data_tour_train)
df_tour

,_id,review_text,지역
0,65ee5e2b901db79521bc8779,[이벤트] 동트는 강원 취향저격 이벤트! 나에게 딱 맞는 강원 네이처 로드 코스는?,강원
1,65ee5e2b901db79521bc877a,봄 오는 소리 귀 기울이고 겨울 가는 아쉬움 녹이고,강원
2,65ee5e2c901db79521bc877b,하얀 이불 벗고 봄으로 가는 길 태평성대 가져온 봉황 눈앞에,강원
3,65ee5e2c901db79521bc877c,"열흘에 한 번, 고원 마을에 열리는 반짝 세일! 태백 통리장",강원
4,65ee5e2c901db79521bc877d,걸음 걸음 쌓이는 설렘 바다로 가는 오름길,강원
...,...,...,...
49657,65ee9fa9c27eda916fc759b4,집에서 만나는 전국 맛집,제주
49658,65ee9faac27eda916fc759b5,[윈터 바캉스] 10만원으로 떠나는 제주 당일치기,제주
49659,65ee9faac27eda916fc759b6,제주 서귀포층 패류 화석산지,제주
49660,65ee9faac27eda916fc759b7,제주 한림 용암동굴지대,제주


In [3]:
from konlpy.tag import Okt
okt = Okt()
stopwords = ['남해','대전','대전','공주','안동','나주','바퀴','보령','고령','신안','특산','공항','곡성','신상','사람','위치','가지','진도','장동','정리','도미','고창','모음','장성','시작','남원','가격','화순','국립','베스트','오토','레이스','후기','피아','중앙','공모전','태안','성주','봉화','남도','목포','군산','구례','순천','문경','완주','청주','청도','여수','울주군','포구','두기','거리','강진','뉴스','방법','통영','부여','한밭수목원','거제','서쪽','동쪽','양산','하동','포항','진주','우도','서귀포','당신', '대관령','속초','특별자치도', '삼척','영월','원주','화천','고성','양양','춘천','양구','철원','홍천','태백','달성군','구석구석','블로그','여행','여행지','경기','여주','서구','현황', '장생포','양평','가창','남평문씨','파주','영상','소개','지역','명소','요즘','마을','달성','한눈','카드','추천','수성','안산','기회','횡성','문진','발생','코스','현풍','문진','만끽']
def Tokenizer(raw, pos=["Noun"], stopword=stopwords):
    word_list = []
    for word, tag in okt.pos(raw,norm=True,stem=True):
        if len(word) > 1 and tag in pos and word not in stopword:
            if mecab.pos(word)[0][1] in ["NNG"]:
                if word == '화관':
                    word = '문화관'
                word_list.append(word)
    return " ".join(word_list)

## 데이터 분류하기

In [4]:
df_tour_train

,_id,concept,season,review_text,지역,review_text_clean
0,65ee6316a64dee66ed301e2a,0,0,[블로그 울산 남구 아이스팩 전용 수거함 어디에 있을까?,울산,아이스팩 전용 수거
1,65ee670ba64dee66ed303175,5,3,[가을여행] 전주 당일치기 여행 BEST10 (feat. 덕진공원팸투어) /가을 단...,전북,가을 전주 당일치기 진공 투어 가을 단풍
2,65ee6a64c528b55b0ae79e58,4,0,"대전 유성구 관평동 맛집, 감동적인 맛과 가격 '매봉 쌀국수와 소바'",대전,맛집 감동 쌀국수
3,65ee60fc901db79521bc991b,3,0,역사를 따라가는 DMZ 로드,경기,역사 로드
4,65ee664fc528b55b0ae78d7a,5,0,"아브라다카브라라는 주문을 외워보고 싶은, '대전 보라매공원크리스마스 풍경'",대전,주문 보라매 공원 풍경
...,...,...,...,...,...,...
9995,65ee9b78c27eda916fc74ead,5,-,서귀포 추천 여행지 신창풍차해안도로,제주,풍차 해안 도로
9996,65ee66b6901db79521bcb1d6,4,-,"반려견과 함께 하는 커피 한 잔, 광주 애견동반카페 모음.ZIP",광주,반려견 커피 애견 동반
9997,65ee6722a64dee66ed3032ef,1,-,전북 문화공간 - 완주 복합문화지구 누에(Nu-e),전북,문화 공간 복합 문화 지구 누에
9998,65ee6cd5c528b55b0ae7a7b4,5,-,"벚꽃은 갔지만 유채꽃 만발, 서울에서 만나는 남해",서울,벚꽃 유채꽃 만발


In [5]:
df_tour_train.isnull().sum()

_id                    0
concept                0
season                 0
review_text            0
지역                     0
review_text_clean    225
dtype: int64

In [6]:
df_tour_train=df_tour_train.dropna()

In [7]:
df_tour_train.isnull().sum()

_id                  0
concept              0
season               0
review_text          0
지역                   0
review_text_clean    0
dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
df_tour_train_train,df_tour_train_test = train_test_split(df_tour_train,test_size=0.3,random_state=42)
df_tour_train['concept'].unique()

array([0, 5, 4, 3, 1, 2, 6])

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfVectorizer = TfidfVectorizer(tokenizer = Tokenizer, use_idf=True,max_df=0.95,min_df=2,ngram_range=(1,2))
tfidfVectorizer.fit(df_tour_train['review_text_clean'])

TfidfVectorizer(max_df=0.95, min_df=2, ngram_range=(1, 2),
                tokenizer=<function Tokenizer at 0x7f24b6a9d120>)

In [10]:
train_feature = tfidfVectorizer.transform(df_tour_train_train['review_text_clean']) # train
concept_train_target = df_tour_train_train['concept'] # test
train_feature_test = tfidfVectorizer.transform(df_tour_train_test['review_text_clean']) # train
concept_train_target_test = df_tour_train_test['concept']


## concept 분류 모델 만들기
- 데이터가 2000개 였을 때 : 0.6003401360544217
- 데이터가 4000개 였을 때 : 0.6909323116219668
- 데이터가 7000개 였을 때 : 0.7263025671944201
- 데이터가 10000개 였을 때 : 0.7494033412887828

In [11]:
from sklearn.ensemble import RandomForestClassifier
randomForestClassifier=RandomForestClassifier(n_jobs=-1)
randomForestClassifier.fit(train_feature,concept_train_target)

RandomForestClassifier(n_jobs=-1)

In [12]:
concept_train_feature_predict = randomForestClassifier.predict(train_feature)
concept_train_feature_predict

array([1, 5, 2, ..., 0, 1, 1])

In [13]:
from sklearn.metrics import classification_report
print(classification_report(concept_train_target,concept_train_feature_predict))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1945
           1       1.00      0.99      1.00      1656
           2       1.00      0.99      1.00       195
           3       0.99      0.99      0.99       325
           4       1.00      1.00      1.00       593
           5       1.00      0.99      1.00      1832
           6       0.99      0.98      0.99       296

    accuracy                           0.99      6842
   macro avg       1.00      0.99      0.99      6842
weighted avg       0.99      0.99      0.99      6842



In [14]:
concept_train_feature_test_predict = randomForestClassifier.predict(train_feature_test)
concept_train_feature_test_predict

array([5, 2, 0, ..., 1, 0, 0])

In [15]:
from sklearn.metrics import classification_report
print(classification_report(concept_train_target_test, concept_train_feature_test_predict))

              precision    recall  f1-score   support

           0       0.68      0.85      0.75       833
           1       0.80      0.70      0.75       729
           2       0.83      0.41      0.55        73
           3       0.86      0.47      0.61       138
           4       0.85      0.73      0.79       232
           5       0.76      0.81      0.79       793
           6       0.68      0.52      0.59       135

    accuracy                           0.75      2933
   macro avg       0.78      0.64      0.69      2933
weighted avg       0.76      0.75      0.75      2933



## season 분류 모델 만들기
- 데이터가 2000개 였을 때 : 0.95
- 데이터가 4000개 였을 때 : 0.96

In [16]:
df_tour_train.query("season != '-'")['season'].unique()

array([0, 3, 4, 1, 2], dtype=object)

In [17]:
df_tour_train_train.query("season != '-'")['review_text_clean']

655           캠핑 구산 해수욕장
178         엄마 학교 신입생 모집
3205      봉유 돼지 짬뽕 전복 짬뽕
800             기념물 이팝나무
92                항공샷 랜선
              ...       
2481         마다 평온 고이 시간
782      벚꽃 폴리스 공원 봄꽃 산책
1723         대비 면역 제철 음식
477     혈관 온라인 건강 캠페인 음식
878       외로움 간직 박물관 비양도
Name: review_text_clean, Length: 2727, dtype: object

In [18]:
season_train_feature = tfidfVectorizer.transform(df_tour_train_train.query("season != '-'")['review_text_clean']) # train
season_train_target = df_tour_train_train.query("season != '-'")['season'].astype('str') # test
season_train_feature_test = tfidfVectorizer.transform(df_tour_train_test.query("season != '-'")['review_text_clean']) # train
season_train_target_test = df_tour_train_test.query("season != '-'")['season'].astype('str') # test

In [19]:
from sklearn.ensemble import RandomForestClassifier
randomForestClassifier=RandomForestClassifier(n_jobs=-1)
randomForestClassifier.fit(season_train_feature,season_train_target)

RandomForestClassifier(n_jobs=-1)

In [20]:
season_train_target_predict = randomForestClassifier.predict(season_train_feature)

In [21]:
from sklearn.metrics import classification_report
print(classification_report(season_train_target, season_train_target_predict))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2273
           1       1.00      1.00      1.00       104
           2       0.99      1.00      1.00       103
           3       1.00      1.00      1.00       148
           4       1.00      1.00      1.00        99

    accuracy                           1.00      2727
   macro avg       1.00      1.00      1.00      2727
weighted avg       1.00      1.00      1.00      2727



In [22]:
season_test_feature_predict = randomForestClassifier.predict(season_train_feature_test)

In [23]:
from sklearn.metrics import classification_report
print(classification_report(season_train_target_test, season_test_feature_predict))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98       966
           1       0.97      0.76      0.85        41
           2       0.98      0.87      0.92        52
           3       0.94      0.83      0.88        81
           4       0.93      0.89      0.91        47

    accuracy                           0.96      1187
   macro avg       0.96      0.87      0.91      1187
weighted avg       0.96      0.96      0.96      1187



## concept 모델 향상시키기

### 모델 선택

#### logisticRegression : 0.75 

In [24]:
logistic_train_feature = tfidfVectorizer.transform(df_tour_train_train['review_text_clean']) # train
logistic_train_target = df_tour_train_train['concept'] # test
logistic_test_feature = tfidfVectorizer.transform(df_tour_train_test['review_text_clean']) # train
logistic_test_target = df_tour_train_test['concept']


In [25]:
from sklearn import linear_model
logisticRegression=linear_model.LogisticRegression()
logisticRegression.fit(logistic_train_feature,logistic_train_target)

LogisticRegression()

In [26]:
logistic_train_feature_predict = logisticRegression.predict(logistic_train_feature)
logistic_train_feature_predict

array([5, 5, 2, ..., 0, 1, 0])

In [27]:
from sklearn.metrics import classification_report
print(classification_report(logistic_train_target,logistic_train_feature_predict))

              precision    recall  f1-score   support

           0       0.77      0.92      0.84      1945
           1       0.86      0.82      0.84      1656
           2       0.97      0.59      0.73       195
           3       0.90      0.57      0.70       325
           4       0.96      0.79      0.86       593
           5       0.85      0.88      0.86      1832
           6       0.82      0.56      0.67       296

    accuracy                           0.83      6842
   macro avg       0.87      0.73      0.79      6842
weighted avg       0.84      0.83      0.83      6842



In [28]:
logistic_test_feature_predict = logisticRegression.predict(logistic_test_feature)
logistic_test_feature_predict

array([5, 5, 0, ..., 1, 0, 1])

In [29]:
from sklearn.metrics import classification_report
print(classification_report(logistic_test_target, logistic_test_feature_predict))

              precision    recall  f1-score   support

           0       0.69      0.85      0.77       833
           1       0.77      0.73      0.75       729
           2       0.94      0.42      0.58        73
           3       0.80      0.41      0.55       138
           4       0.88      0.69      0.77       232
           5       0.76      0.82      0.79       793
           6       0.75      0.41      0.53       135

    accuracy                           0.75      2933
   macro avg       0.80      0.62      0.68      2933
weighted avg       0.76      0.75      0.74      2933



#### Decision Tree Classifer 0.67 

In [30]:
DTC_train_feature = tfidfVectorizer.transform(df_tour_train_train['review_text_clean']) # train
DTC_train_target = df_tour_train_train['concept'] # test
DTC_test_feature = tfidfVectorizer.transform(df_tour_train_test['review_text_clean']) # train
DTC_test_target = df_tour_train_test['concept']


In [31]:
from sklearn.tree import DecisionTreeClassifier
decisionTreeClassifier=DecisionTreeClassifier()
decisionTreeClassifier.fit(DTC_train_feature,DTC_train_target)

DecisionTreeClassifier()

In [32]:
DTC_train_feature_predict = decisionTreeClassifier.predict(DTC_train_feature)
DTC_train_feature_predict

array([1, 5, 2, ..., 0, 1, 1])

In [33]:
from sklearn.metrics import classification_report
print(classification_report(DTC_train_target,DTC_train_feature_predict))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1945
           1       1.00      1.00      1.00      1656
           2       1.00      0.99      1.00       195
           3       1.00      0.99      0.99       325
           4       1.00      1.00      1.00       593
           5       1.00      0.99      1.00      1832
           6       1.00      0.97      0.99       296

    accuracy                           0.99      6842
   macro avg       1.00      0.99      0.99      6842
weighted avg       0.99      0.99      0.99      6842



In [34]:
DTC_test_feature_predict = decisionTreeClassifier.predict(DTC_test_feature)
DTC_test_feature_predict

array([0, 5, 6, ..., 1, 5, 0])

In [35]:
from sklearn.metrics import classification_report
print(classification_report(DTC_test_target, DTC_test_feature_predict))

              precision    recall  f1-score   support

           0       0.66      0.70      0.68       833
           1       0.70      0.67      0.68       729
           2       0.48      0.38      0.43        73
           3       0.51      0.45      0.48       138
           4       0.77      0.72      0.74       232
           5       0.68      0.70      0.69       793
           6       0.48      0.49      0.49       135

    accuracy                           0.67      2933
   macro avg       0.61      0.59      0.60      2933
weighted avg       0.66      0.67      0.66      2933



#### Naive Bayes 0.41

In [36]:
nb_train_feature = tfidfVectorizer.transform(df_tour_train_train['review_text_clean']) # train
nb_train_target = df_tour_train_train['concept'] # test
nb_test_feature = tfidfVectorizer.transform(df_tour_train_test['review_text_clean']) # train
nb_test_target = df_tour_train_test['concept']


In [37]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(nb_train_feature.toarray(),nb_train_target)

GaussianNB()

In [38]:
nb_train_feature_predict = nb.predict(nb_train_feature.toarray())
nb_train_feature_predict

array([2, 4, 2, ..., 0, 5, 1])

In [39]:
from sklearn.metrics import classification_report
print(classification_report(nb_train_target,nb_train_feature_predict))

              precision    recall  f1-score   support

           0       1.00      0.48      0.65      1945
           1       0.83      0.67      0.74      1656
           2       0.31      1.00      0.47       195
           3       0.53      0.94      0.68       325
           4       0.68      0.96      0.79       593
           5       0.70      0.74      0.72      1832
           6       0.43      0.86      0.57       296

    accuracy                           0.69      6842
   macro avg       0.64      0.81      0.66      6842
weighted avg       0.78      0.69      0.69      6842



In [40]:
nb_test_feature_predict = nb.predict(nb_test_feature.toarray())
nb_test_feature_predict

array([2, 1, 0, ..., 1, 3, 1])

In [41]:
from sklearn.metrics import classification_report
print(classification_report(nb_test_target, nb_test_feature_predict))

              precision    recall  f1-score   support

           0       0.53      0.31      0.39       833
           1       0.45      0.39      0.42       729
           2       0.07      0.25      0.11        73
           3       0.29      0.43      0.35       138
           4       0.46      0.63      0.53       232
           5       0.47      0.48      0.48       793
           6       0.17      0.28      0.21       135

    accuracy                           0.41      2933
   macro avg       0.35      0.40      0.36      2933
weighted avg       0.45      0.41      0.42      2933



#### K-Nearest Neighbors : 0.70

In [42]:
knn_train_feature = tfidfVectorizer.transform(df_tour_train_train['review_text_clean']) # train
knn_train_target = df_tour_train_train['concept'] # test
knn_test_feature = tfidfVectorizer.transform(df_tour_train_test['review_text_clean']) # train
knn_test_target = df_tour_train_test['concept']


In [43]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(knn_train_feature,knn_train_target)

KNeighborsClassifier()

In [44]:
knn_train_feature_predict = knn.predict(knn_train_feature)
knn_train_feature_predict

array([1, 5, 2, ..., 0, 0, 0])

In [45]:
from sklearn.metrics import classification_report
print(classification_report(knn_train_target,knn_train_feature_predict))

              precision    recall  f1-score   support

           0       0.73      0.83      0.78      1945
           1       0.78      0.75      0.77      1656
           2       0.81      0.71      0.75       195
           3       0.77      0.64      0.70       325
           4       0.91      0.78      0.84       593
           5       0.79      0.81      0.80      1832
           6       0.69      0.52      0.60       296

    accuracy                           0.78      6842
   macro avg       0.78      0.72      0.75      6842
weighted avg       0.78      0.78      0.78      6842



In [46]:
knn_test_feature_predict = knn.predict(knn_test_feature)
knn_test_feature_predict

array([5, 5, 1, ..., 1, 5, 1])

In [47]:
from sklearn.metrics import classification_report
print(classification_report(knn_test_target, knn_test_feature_predict))

              precision    recall  f1-score   support

           0       0.65      0.73      0.69       833
           1       0.70      0.70      0.70       729
           2       0.71      0.55      0.62        73
           3       0.72      0.53      0.61       138
           4       0.86      0.67      0.76       232
           5       0.71      0.76      0.73       793
           6       0.60      0.43      0.50       135

    accuracy                           0.70      2933
   macro avg       0.71      0.62      0.66      2933
weighted avg       0.70      0.70      0.69      2933



#### RandomForestClassifier : 0.75 

In [48]:
RFC_train_feature = tfidfVectorizer.transform(df_tour_train_train['review_text_clean']) # train
RFC_train_target = df_tour_train_train['concept'] # test
RFC_test_feature = tfidfVectorizer.transform(df_tour_train_test['review_text_clean']) # train
RFC_test_target = df_tour_train_test['concept']


In [49]:
from sklearn.ensemble import RandomForestClassifier
randomForestClassifier=RandomForestClassifier(n_jobs=-1)
randomForestClassifier.fit(RFC_train_feature,RFC_train_target)

RandomForestClassifier(n_jobs=-1)

In [50]:
RFC_train_feature_predict = randomForestClassifier.predict(RFC_train_feature)
RFC_train_feature_predict

array([1, 5, 2, ..., 0, 1, 1])

In [51]:
from sklearn.metrics import classification_report
print(classification_report(RFC_train_target,RFC_train_feature_predict))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1945
           1       1.00      0.99      1.00      1656
           2       0.99      1.00      1.00       195
           3       0.99      0.99      0.99       325
           4       1.00      1.00      1.00       593
           5       1.00      1.00      1.00      1832
           6       0.99      0.99      0.99       296

    accuracy                           0.99      6842
   macro avg       0.99      0.99      0.99      6842
weighted avg       0.99      0.99      0.99      6842



In [52]:
RFC_test_feature_predict = randomForestClassifier.predict(RFC_test_feature)
RFC_test_feature_predict

array([5, 2, 0, ..., 1, 0, 1])

In [53]:
from sklearn.metrics import classification_report
print(classification_report(RFC_test_target, RFC_test_feature_predict))

              precision    recall  f1-score   support

           0       0.69      0.85      0.76       833
           1       0.81      0.72      0.76       729
           2       0.85      0.40      0.54        73
           3       0.83      0.45      0.58       138
           4       0.87      0.72      0.79       232
           5       0.76      0.81      0.79       793
           6       0.69      0.53      0.60       135

    accuracy                           0.75      2933
   macro avg       0.78      0.64      0.69      2933
weighted avg       0.76      0.75      0.75      2933



#### Support Vector Machine : 0.76

In [54]:
SVM_train_feature = tfidfVectorizer.transform(df_tour_train_train['review_text_clean']) # train
SVM_train_target = df_tour_train_train['concept'] # test
SVM_test_feature = tfidfVectorizer.transform(df_tour_train_test['review_text_clean']) # train
SVM_test_target = df_tour_train_test['concept']


In [55]:
from sklearn import svm
svm_model = svm.SVC()
svm_model.fit(SVM_train_feature,SVM_train_target)

SVC()

In [56]:
svm_model.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [57]:
SVM_train_feature_predict = svm_model.predict(SVM_train_feature)
SVM_train_feature_predict

array([1, 5, 2, ..., 0, 1, 1])

In [58]:
from sklearn.metrics import classification_report
print(classification_report(SVM_train_target,SVM_train_feature_predict))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      1945
           1       0.95      0.94      0.94      1656
           2       0.99      0.86      0.92       195
           3       0.95      0.86      0.90       325
           4       0.98      0.95      0.97       593
           5       0.95      0.96      0.95      1832
           6       0.95      0.79      0.86       296

    accuracy                           0.94      6842
   macro avg       0.95      0.90      0.93      6842
weighted avg       0.94      0.94      0.94      6842



In [59]:
SVM_test_feature_predict = svm_model.predict(SVM_test_feature)
SVM_test_feature_predict

array([5, 5, 0, ..., 1, 5, 0])

In [60]:
from sklearn.metrics import classification_report
print(classification_report(SVM_test_target, SVM_test_feature_predict))

              precision    recall  f1-score   support

           0       0.68      0.88      0.77       833
           1       0.80      0.72      0.76       729
           2       0.92      0.48      0.63        73
           3       0.82      0.48      0.61       138
           4       0.91      0.69      0.78       232
           5       0.79      0.82      0.80       793
           6       0.75      0.45      0.56       135

    accuracy                           0.76      2933
   macro avg       0.81      0.65      0.70      2933
weighted avg       0.77      0.76      0.76      2933



### hyper parameter tuning

#### Support Vector Machine
* kernel : liner, gamma : 0.1, C : 1, degree : 2
* accuracy : 0.77

In [141]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.metrics import f1_score, make_scorer

hyper_parameters = {'kernel': ['linear'], 'gamma': [0.1], 'C':[1],'degree':[2]}
svc_estimator_model = svm.SVC()

# score 방식 지정
scoring =  make_scorer(f1_score)
svc_grid_model = GridSearchCV(svc_estimator_model,hyper_parameters, scoring = scoring, cv=5) # 데이터셋 폴드

svc_grid_model

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1], 'degree': [2], 'gamma': [0.1],
                         'kernel': ['linear']},
             scoring=make_scorer(f1_score, response_method='predict'))

In [142]:
svc_grid_model.fit(SVM_train_feature,SVM_train_target)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1], 'degree': [2], 'gamma': [0.1],
                         'kernel': ['linear']},
             scoring=make_scorer(f1_score, response_method='predict'))

In [143]:
svc_best_model = svc_grid_model.best_estimator_
svc_best_model

SVC(C=1, degree=2, gamma=0.1, kernel='linear')

In [144]:
svc_best_model.get_params()

{'C': 1,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 2,
 'gamma': 0.1,
 'kernel': 'linear',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [145]:
SVM_test_feature_predict = svc_best_model.predict(SVM_test_feature)
SVM_test_feature_predict

array([5, 5, 0, ..., 1, 5, 1])

In [146]:
from sklearn.metrics import classification_report
print(classification_report(SVM_test_target, SVM_test_feature_predict))

              precision    recall  f1-score   support

           0       0.71      0.85      0.77       833
           1       0.80      0.72      0.76       729
           2       0.87      0.55      0.67        73
           3       0.79      0.55      0.65       138
           4       0.89      0.75      0.81       232
           5       0.78      0.83      0.80       793
           6       0.70      0.53      0.60       135

    accuracy                           0.77      2933
   macro avg       0.79      0.68      0.72      2933
weighted avg       0.77      0.77      0.76      2933



#### logisticRegression
* penalty : l2, solver : newton-cg, C : 0.1, class_weight : None
* accuracy : 0.75

In [218]:
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
logisticRegression
from sklearn.metrics import f1_score, make_scorer

hyper_parameters = {'penalty': ['l2'],'solver':['newton-cg'],'C':[0.1],'class_weight':[None]}
lr_estimator_model = linear_model.LogisticRegression()

# score 방식 지정
scoring =  make_scorer(f1_score)
lr_grid_model = GridSearchCV(lr_estimator_model,hyper_parameters, scoring = scoring, cv=5) # 데이터셋 폴드

lr_grid_model

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.1], 'class_weight': [None], 'penalty': ['l2'],
                         'solver': ['newton-cg']},
             scoring=make_scorer(f1_score, response_method='predict'))

In [219]:
lr_grid_model.fit(logistic_train_feature,logistic_train_target)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.1], 'class_weight': [None], 'penalty': ['l2'],
                         'solver': ['newton-cg']},
             scoring=make_scorer(f1_score, response_method='predict'))

In [220]:
lr_best_model = lr_grid_model.best_estimator_
lr_best_model

LogisticRegression(C=0.1, solver='newton-cg')

In [221]:
lr_best_model.get_params()

{'C': 0.1,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'newton-cg',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [222]:
lr_test_feature_predict = lr_best_model.predict(logistic_test_feature)
lr_test_feature_predict

array([5, 5, 0, ..., 1, 0, 0])

In [223]:
from sklearn.metrics import classification_report
print(classification_report(logistic_test_target, logistic_test_feature_predict))

              precision    recall  f1-score   support

           0       0.69      0.85      0.77       833
           1       0.77      0.73      0.75       729
           2       0.94      0.42      0.58        73
           3       0.80      0.41      0.55       138
           4       0.88      0.69      0.77       232
           5       0.76      0.82      0.79       793
           6       0.75      0.41      0.53       135

    accuracy                           0.75      2933
   macro avg       0.80      0.62      0.68      2933
weighted avg       0.76      0.75      0.74      2933



#### Support Vector Machine
* n_jobs : -1, n_estimators : 100, criterion : gini, max_depth : None, max_features : sqrt
* accuracy : 0.75

In [329]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, make_scorer

hyper_parameters = {'n_jobs':[-1],'n_estimators':[100],'criterion':['gini'],'max_depth':[None],'max_features':['sqrt']}
RFC_estimator_model =RandomForestClassifier()

# score 방식 지정
scoring =  make_scorer(f1_score)
RFC_grid_model = GridSearchCV(RFC_estimator_model,hyper_parameters, scoring = scoring, cv=5) # 데이터셋 폴드

RFC_grid_model

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini'], 'max_depth': [None],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [100], 'n_jobs': [-1]},
             scoring=make_scorer(f1_score, response_method='predict'))

In [330]:
RFC_grid_model.fit(RFC_train_feature,RFC_train_target)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini'], 'max_depth': [None],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [100], 'n_jobs': [-1]},
             scoring=make_scorer(f1_score, response_method='predict'))

In [331]:
RFC_best_model = RFC_grid_model.best_estimator_
RFC_best_model

RandomForestClassifier(n_jobs=-1)

In [332]:
RFC_best_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [333]:
RFC_test_feature_predict = RFC_best_model.predict(RFC_test_feature)
RFC_test_feature_predict

array([5, 5, 0, ..., 1, 0, 1])

In [334]:
from sklearn.metrics import classification_report
print(classification_report(RFC_test_target, RFC_test_feature_predict))

              precision    recall  f1-score   support

           0       0.68      0.85      0.75       833
           1       0.80      0.70      0.75       729
           2       0.91      0.40      0.55        73
           3       0.83      0.46      0.59       138
           4       0.86      0.70      0.77       232
           5       0.75      0.82      0.79       793
           6       0.67      0.51      0.58       135

    accuracy                           0.75      2933
   macro avg       0.79      0.63      0.68      2933
weighted avg       0.76      0.75      0.74      2933



## 피클 내보내기

In [335]:
import pickle
with open('svc_best_model_jh.pkl','wb') as pickle_file:   # pickle에 저장
  pickle.dump(obj=svc_best_model,file=pickle_file)